<a href="https://colab.research.google.com/github/sarvnn/EVA4/blob/master/EVA4_Assignment4_Accuracy_99_39.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input - 28x28x3 OUtput - 28x28x32 
        self.bnm1 = nn.BatchNorm2d(32, momentum=0.9)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.dropout1 = nn.Dropout(p=0.05, inplace=False)
        self.bnm2 = nn.BatchNorm2d(64, momentum=0.9)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bnm3 = nn.BatchNorm2d(128, momentum=0.9)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.dropout2 = nn.Dropout(p=0.05, inplace=False)
        self.bnm4 = nn.BatchNorm2d(256, momentum=0.9)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.bnm5 = nn.BatchNorm2d(512, momentum=0.9)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.dropout3 = nn.Dropout(p=0.05, inplace=False)
        self.bnm6 = nn.BatchNorm2d(1024, momentum=0.9)
        self.conv7 = nn.Conv2d(1024, 10, 3)


    def forward(self, x):
        x = self.pool1(F.relu(self.bnm2(self.dropout1(self.conv2(F.relu(self.bnm1(self.conv1(x))))))))
        x = self.pool2(F.relu(self.bnm4(self.dropout2(self.conv4(F.relu(self.bnm3(self.conv3(x))))))))
        x = F.relu(self.bnm6(self.dropout3(self.conv6(F.relu(self.bnm5(self.conv5(x)))))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
            Conv2d-3           [-1, 64, 28, 28]          18,496
           Dropout-4           [-1, 64, 28, 28]               0
       BatchNorm2d-5           [-1, 64, 28, 28]             128
         MaxPool2d-6           [-1, 64, 14, 14]               0
            Conv2d-7          [-1, 128, 14, 14]          73,856
       BatchNorm2d-8          [-1, 128, 14, 14]             256
            Conv2d-9          [-1, 256, 14, 14]         295,168
          Dropout-10          [-1, 256, 14, 14]               0
      BatchNorm2d-11          [-1, 256, 14, 14]             512
        MaxPool2d-12            [-1, 256, 7, 7]               0
           Conv2d-13            [-1, 512, 5, 5]       1,180,160
      BatchNorm2d-14            [-1, 51

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 256

kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0008, momentum=0.9)

for epoch in range(1, 21):
    print("ëpoch = ", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/235 [00:00<?, ?it/s]

ëpoch =  1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/235 [00:00<?, ?it/s]


Test set: Average loss: 0.3414, Accuracy: 8799/10000 (88%)

ëpoch =  2


  0%|          | 0/235 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0208, Accuracy: 9936/10000 (99%)

ëpoch =  19


  0%|          | 0/235 [00:00<?, ?it/s]


Test set: Average loss: 0.0204, Accuracy: 9939/10000 (99%)

ëpoch =  20


loss=0.0044935643672943115 batch_id=234: 100%|██████████| 235/235 [00:11<00:00, 21.60it/s]



Test set: Average loss: 0.0218, Accuracy: 9935/10000 (99%)

